# Model Evaluation of NeoPrecis-Immuno

This notebook is for evaluating immunogenicity prediction. 

The data is presented in Table S2-3, and the results are shown in Figure 2-3.

In [ ]:
from imm_utils import *

# model color map
models = ['NP-Immuno', 'NetMHCpan', 'DeepNeo', 'PRIME', 'ICERFIRE']
metric_color_dict = {models[i]: sns.color_palette('pastel')[i] for i in range(len(models))}
metric_color_dict['PHBR'] = metric_color_dict['NetMHCpan']
metric_color_dict['NetMHCIIpan'] = metric_color_dict['NetMHCpan']

### Validation on the CEDAR immunogenicity dataset

In [ ]:
# loading CEDAR data

cedar_file = 'TableS2.csv' # fill in the path of TableS2.csv
cedar_df = pd.read_csv(cedar_file)

In [ ]:
# evaluation with bootstrapping (only MHC-I)

x_col_dict = {
    'MT_netMHCpan_rank': 'less',
    'PRIME': 'less',
    'ICERFIRE': 'less',
    'DeepNeo': 'greater',
    'NeoPrecis-Immuno': 'greater'
}
y_col = 'label'

# testing set
test_df = cedar_df[(cedar_df[['in_prime', 'in_icerfire', 'in_deepneo']] == False).all(axis=1)] # not present in other predictors' training set
test_df = test_df[~test_df['dataset'].isin(['train', 'valid'])] # not in our training set
test_df = test_df[test_df['MHC']=='I'] # only MHC-I
test_df = test_df.dropna(subset=['PRIME', 'ICERFIRE', 'DeepNeo'])
print('#Samples =', test_df.shape[0])

# result
perf_df = Performance(test_df, x_col_dict, y_col, fillna=False) # performance
print(perf_df)

# bootstrapping
bstp_df = Bootstrapping(test_df, x_col_dict, y_col, fillna=False, n_iter=1000)

In [ ]:
# plot - AUROC

metric = 'AUROC'

# df
plot_df = bstp_df.copy()
plot_df['Model'] = plot_df['Model'].replace({'MT_netMHCpan_rank': 'NetMHCpan', 'NeoPrecis-Immuno': 'NP-Immuno'})

# plot
order_list = ['NP-Immuno', 'NetMHCpan', 'DeepNeo', 'PRIME', 'ICERFIRE']
fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=dpi)
g = sns.boxplot(data=plot_df, x='Model', y=metric, order=order_list, ax=ax)
ax.tick_params(axis='x', rotation=60)
_ = ax.set_xlabel('')

# color
for i, patch in enumerate(g.patches):
    patch.set_facecolor(metric_color_dict[order_list[i]])

# save
fig.tight_layout()

In [ ]:
# plot - AUPRC

metric = 'AUPRC'

# df
plot_df = bstp_df.copy()
plot_df['Model'] = plot_df['Model'].replace({'MT_netMHCpan_rank': 'NetMHCpan', 'NeoPrecis-Immuno': 'NP-Immuno'})

# plot
order_list = ['NP-Immuno', 'NetMHCpan', 'DeepNeo', 'PRIME', 'ICERFIRE']
fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=dpi)
g = sns.boxplot(data=plot_df, x='Model', y=metric, order=order_list, ax=ax)
ax.tick_params(axis='x', rotation=60)
_ = ax.set_xlabel('')

# color
for i, patch in enumerate(g.patches):
    patch.set_facecolor(metric_color_dict[order_list[i]])

# save
fig.tight_layout()

### Validation on the NCI dataset

In [ ]:
# load NCI data

nci_file = 'TableS3.csv'  # fill in the path of TableS3.csv
nci_df = pd.read_csv(nci_file)

In [ ]:
# filtered by abundance and presentation
def NCI_comparison(mut_df, mhc, x_cols, y_col, dropna=False):
    # x_cols
    x_col_dict = {f'{col}-{mhc}': ('less' if col in ['PHBR', 'PRIME', 'ICERFIRE'] else 'greater') for col in x_cols}
    x_col_dict = {k:v for k,v in x_col_dict.items() if k in mut_df.columns}

    # filtering
    print('#Samples =', mut_df.shape[0])
    filter_mut_df = mut_df[(mut_df['DNA_AF']>0) & (mut_df['RNA_AF']>0) & (mut_df['RNA_EXP_QRT']>1)] # filter by expression
    print('Filtered by abundance =', filter_mut_df.shape[0])
    filter_mut_df = filter_mut_df[filter_mut_df['Consequence']=='missense_variant'] # focus on substitutions
    print('Filtered by SNVs =', filter_mut_df.shape[0])
    filter_mut_df = filter_mut_df.dropna(subset=[f'PHBR-{mhc}']) # drop invalid mutations
    print('Filtered by valid PHBR =', filter_mut_df.shape[0])
    filter_mut_df = filter_mut_df[filter_mut_df[f'Robustness-{mhc}']>0]
    print('Filtered by robustness>0 =', filter_mut_df.shape[0])
    if dropna:
        cols = list(x_col_dict.keys()) + [y_col,]
        filter_mut_df = filter_mut_df.dropna(subset=cols) # drop NA
        print('Filtered by NA =', filter_mut_df.shape[0])

    # validation
    results = Performance(filter_mut_df, x_col_dict, y_col, fillna=True)

    return filter_mut_df, results

In [ ]:
### comparing with other methods
# drop NA

methods = ['NP-Immuno', 'PHBR', 'DeepNeo', 'PRIME', 'ICERFIRE']

print('===MHCI===')
mhci_filter_mut_df, mhci_results = NCI_comparison(nci_df, 'I', methods, 'CD8', dropna=True) # drop NA to ensure all samples with predictions
print('===MHCII===')
mhcii_filter_mut_df, mhcii_results = NCI_comparison(nci_df, 'II', methods, 'CD4', dropna=True)

# AUROC
metric = 'AUROC'
TwoPerfBarPlot(mhci_results, mhcii_results, methods, metric, palette=metric_color_dict, hue_order=methods, figsize=(4,3))

# AUPRC
metric = 'AUPRC'
TwoPerfBarPlot(mhci_results, mhcii_results, methods, metric, palette=metric_color_dict, hue_order=methods, figsize=(4,3))

In [ ]:
### component contribution
# fill NA with 0

methods = ['BLOSUMDist', 'SubDist', 'SubPosDist', 'GeoDist', 'CRD']
metric = 'AUROC'

print('===MHCI===')
mhci_filter_mut_df, mhci_results = NCI_comparison(nci_df, 'I', methods, 'CD8', dropna=False)
print('===MHCII===')
mhcii_filter_mut_df, mhcii_results = NCI_comparison(nci_df, 'II', methods, 'CD4', dropna=False)

TwoPerfBarPlot(mhci_results, mhcii_results, methods, metric, palette='pastel', figsize=(7,3), annot=True)